# Database Load

In [1]:
from flask import Flask, jsonify
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.dialects.postgresql import JSON
from flask import render_template
import csv
import json
import glob
import psycopg2
import os

app = Flask(__name__)

app.config['DEBUG'] = True
DB_URL = os.environ['DATABASE_URL']

app.config['SQLALCHEMY_DATABASE_URI'] = DB_URL
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

In [2]:
class Countries (db.Model):
    """Model for the countries table"""
    __tablename__ = 'countries4'

    id = db.Column(db.Integer, primary_key = True)
    iso_a3 = db.Column(db.String())
    country_name = db.Column(db.String())
    indicator_code = db.Column(db.String())
    year = db.Column(db.Integer)
    value = db.Column(db.String())


In [3]:
db.drop_all()
db.create_all()

In [4]:
# Get all .csv files in ../Output_Data directory
csvlist = [f for f in glob.glob("../Output_data/*.csv")]
csvlist

['../Output_data\\AccessToElectricity.csv',
 '../Output_data\\Births attended by skilled health staff.csv',
 '../Output_data\\Children out of school, primary, female.csv',
 '../Output_data\\corruption_final_version_combined.csv',
 '../Output_data\\country_codes.csv',
 '../Output_data\\Crude birth rate.csv',
 '../Output_data\\Crude Death rate.csv',
 '../Output_data\\Current health expenditure (% of GDP).csv',
 '../Output_data\\Current health expenditure per capita, PPP (current international $).csv',
 '../Output_data\\econ_freedom.csv',
 '../Output_data\\Fertility rate, total (births per woman).csv',
 '../Output_data\\Fixed broadband subscriptions (per 100 people).csv',
 '../Output_data\\healthcare_coverage.csv',
 '../Output_data\\Hospital beds (per 1,000 people).csv',
 '../Output_data\\human-rights-scores.csv',
 '../Output_data\\InfantMortalityRate.csv',
 '../Output_data\\LifeExpectancy.csv',
 '../Output_data\\LifeExpectancyMale.csv',
 '../Output_data\\LiteracyAdultFemale.csv',
 '../Ou

In [5]:
alldata = []
for csvfile in csvlist:
    with open(csvfile, encoding='utf-8-sig') as f:
        reader = csv.reader(f)
        newdata = [row for row in reader]
        alldata.append(newdata)
len(alldata)

25

In [24]:
for n in range(len(alldata)):
    print(alldata[20][n][3])

Indicator_Code
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP
MEDS.PERPOP


In [6]:
def get_grid_type(grid):
    if 'Year' in grid[0]:
        return 'row_by_year'
    else:
        for year_to_check in range(1960, 2999):
            if str(year_to_check) in grid[0]:
                return 'col_by_year'
            elif (str(year_to_check) + '.0') in grid[0]:
                return 'col_by_year'
    print('Undetermined grid')
    return 'undetermined'

In [7]:
def find_key_cols_for_by_year(grid):
    started_flag = False
    country_name_ix = -1
    country_code_ix = -1
    indicator_code_ix = -1
    first_year_ix = -1
    last_year_ix = -1
    num_cols = len(grid[0])
    for col_ix in range(num_cols):
        if grid[0][col_ix] == 'Country':
            country_name_ix = col_ix
        elif grid[0][col_ix] == 'Entity':
            country_name_ix = col_ix
        elif grid[0][col_ix] == 'Country_Name':
            country_name_ix = col_ix
        elif grid[0][col_ix] == 'Country Name':
            country_name_ix = col_ix
        elif grid[0][col_ix] == 'Country_Code':
            country_code_ix = col_ix
        elif grid[0][col_ix] == 'Country Code':
            country_code_ix = col_ix
        elif grid[0][col_ix] == 'Country ISO3':
            country_code_ix = col_ix
        elif grid[0][col_ix] == 'Code':
            country_code_ix = col_ix
        elif grid[0][col_ix] == 'Indicator_Code':
            indicator_code_ix = col_ix
        elif grid[0][col_ix] == 'Indicator Code':
            indicator_code_ix = col_ix
        elif grid[0][col_ix] == 'Indicator Id':
            indicator_code_ix = col_ix
        else:
            try:
                year_col = int(float(grid[0][col_ix]))
            except ValueError:
                pass
            else:
                if (year_col > 1960) and (year_col < 2999):
                    if started_flag:
                        last_year_ix = col_ix
                    else:
                        first_year_ix = col_ix
                        last_year_ix = col_ix
                        started_flag = True
    return country_name_ix, country_code_ix, indicator_code_ix, first_year_ix, last_year_ix

In [8]:
def find_key_cols_for_by_row(grid):
    country_name_ix = -1
    country_code_ix = -1
    indicator_code_ix = -1
    year_ix = -1
    num_cols = len(grid[0])
    for col_ix in range(num_cols):
        if grid[0][col_ix] == 'Country':
            country_name_ix = col_ix
        elif grid[0][col_ix] == 'Entity':
            country_name_ix = col_ix
        elif grid[0][col_ix] == 'Country_Name':
            country_name_ix = col_ix
        elif grid[0][col_ix] == 'Country Name':
            country_name_ix = col_ix
        elif grid[0][col_ix] == 'Country_Code':
            country_code_ix = col_ix
        elif grid[0][col_ix] == 'Country Code':
            country_code_ix = col_ix
        elif grid[0][col_ix] == 'Country ISO3':
            country_code_ix = col_ix
        elif grid[0][col_ix] == 'Code':
            country_code_ix = col_ix
        elif grid[0][col_ix] == 'Indicator_Code':
            indicator_code_ix = col_ix
        elif grid[0][col_ix] == 'Indicator Code':
            indicator_code_ix = col_ix
        elif grid[0][col_ix] == 'Indicator Id':
            indicator_code_ix = col_ix
        elif grid[0][col_ix] == 'Year':
            year_ix = col_ix
    return country_name_ix, country_code_ix, indicator_code_ix, year_ix

In [9]:
def process_by_year(grid, global_id):
    added_count = 0
    data_to_add = set()
    cname_col, ccode_col, icode_col, firstyr_col, lastyr_col = find_key_cols_for_by_year(grid)
    if (cname_col < 0) or (ccode_col < 0) or (icode_col < 0) or (firstyr_col < 0) or (lastyr_col < 0):
        print('Unable to parse csv data')
        print(cname_col)
        print(ccode_col)
        print(icode_col)
        print(firstyr_col)
        print(lastyr_col)
        return set()
    at_header = True
    for row in grid:
        if at_header:
            at_header = False
            continue
        else:
            ccode = row[ccode_col]
            cname = row[cname_col]
            icode = row[icode_col]
            for year_ix in range(firstyr_col, lastyr_col + 1):
                iyear = int(float(grid[0][year_ix]))
                ivalue = row[year_ix]
                if ivalue:
                    data_to_add.add(icode)
#                    db.session.add(data_to_add)
#                    db.session.commit
                    global_id += 1
                    added_count += 1
    return data_to_add

In [10]:
def process_by_row(grid, global_id):
    added_count = 0
    data_to_add = set()
    cname_col, ccode_col, icode_col, yr_col = find_key_cols_for_by_row(grid)
    if (cname_col < 0) or (ccode_col < 0) or (icode_col < 0) or (yr_col < 0):
        print('Unable to parse csv data')
        print(cname_col)
        print(ccode_col)
        print(icode_col)
        print(yr_col)
        return set()
    at_header = True
    for row in grid:
        if at_header:
            at_header = False
            continue
        else:
            ccode = row[ccode_col]
            cname = row[cname_col]
            icode = row[icode_col]
            iyear = int(float(row[yr_col]))
            ivalue = row[yr_col + 1]
            if ivalue:
                data_to_add.add(icode)
#                db.session.add(data_to_add)
#               db.session.commit
                global_id += 1
                added_count += 1
    return data_to_add

In [13]:
global_id = 0
icodes = set()
num_added = 0
for two_d_grid in alldata:
    grid_type = get_grid_type(two_d_grid)
    if grid_type == 'col_by_year':
        dataset = process_by_year(two_d_grid, global_id)
        icodes = icodes|dataset
        print('Grid with multiple years processed: added ' + str(num_added))
        if 'Indicator Code' in icodes:
            print('Added Indicator Code')
        global_id += num_added
    elif grid_type == 'row_by_year':
        dataset = process_by_row(two_d_grid, global_id)
        icodes = icodes|dataset
        if 'Indicator Code' in icodes:
            print('Added Indicator Code')
        print('Grid with one year per row processed: added ' + str(num_added))
        global_id += num_added

Grid with multiple years processed: added 0
Grid with multiple years processed: added 0
Grid with multiple years processed: added 0
Grid with multiple years processed: added 0
Undetermined grid
Grid with multiple years processed: added 0
Grid with multiple years processed: added 0
Grid with multiple years processed: added 0
Grid with multiple years processed: added 0
Grid with multiple years processed: added 0
Grid with multiple years processed: added 0
Grid with multiple years processed: added 0
Grid with one year per row processed: added 0
Grid with multiple years processed: added 0
Grid with one year per row processed: added 0
Grid with multiple years processed: added 0
Grid with multiple years processed: added 0
Grid with multiple years processed: added 0
Grid with multiple years processed: added 0
Grid with multiple years processed: added 0
Grid with one year per row processed: added 0
Grid with multiple years processed: added 0
Grid with multiple years processed: added 0
Added In

In [19]:
test_result = Countries.query.filter_by(indicator_code ='SH.STA.BRTC.ZS').order_by(Countries.iso_a3).all()

In [20]:
len(test_result)

2548

In [12]:
icodes

{'747',
 '748',
 '749',
 '750',
 '751',
 '752',
 '753',
 '754',
 '755',
 '756',
 '757',
 '758',
 '759',
 'CORRUP.HOM',
 'EG.ELC.ACCS.ZS',
 'HLTH.INS',
 'HRP.SCORE',
 'IT.NET.BBND.P2',
 'Indicator Code',
 'MEDS.PERPOP',
 'SE.ADT.1524.LT.FE.ZS',
 'SE.ADT.LITR.FE.ZS',
 'SE.PRM.PRSL.FE.ZS',
 'SE.PRM.PRSL.MA.ZS',
 'SE.PRM.UNER.FE',
 'SH.MED.BEDS.ZS',
 'SH.STA.BRTC.ZS',
 'SH.XPD.CHEX.GD.ZS',
 'SH.XPD.CHEX.PP.CD',
 'SH.XPD.OOPC.PC.CD',
 'SM.POP.NETM',
 'SP.DYN.CBRT.IN',
 'SP.DYN.CDRT.IN',
 'SP.DYN.IMRT.IN',
 'SP.DYN.LE00.IN',
 'SP.DYN.LE00.MA.IN',
 'SP.DYN.TFRT.IN'}